In [1]:
import platform
print(platform.python_version())

3.8.10


In [ ]:
import tensorflow as tf

In [4]:
import os
import numpy as np
import pandas as pd

In [8]:
train_imgs = os.listdir("./data/tfrecords-jpeg-192x192/train")
test_imgs = os.listdir("./data/tfrecords-jpeg-192x192/test")
val_imgs = os.listdir("./data/tfrecords-jpeg-192x192/val")
print(len(train_imgs),len(test_imgs),len(val_imgs))

16 16 16


In [ ]:
import pandas as pd
from PIL import Image
import tensorflow as tf
import io

import torch
import torch.nn as nn
from torch import Tensor
import torch.nn.functional as F
from torch.utils.data import Dataset , DataLoader
from torchvision import transforms , models
from torch.optim import SGD

import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

In [ ]:
train_feature_description = {
    'class': tf.io.FixedLenFeature([], tf.int64),
    'id': tf.io.FixedLenFeature([], tf.string),
    'image': tf.io.FixedLenFeature([], tf.string),
}
test_feature_description = {
    'id': tf.io.FixedLenFeature([], tf.string),
    'image': tf.io.FixedLenFeature([], tf.string),
}

In [ ]:
def _parse_image_function(example_proto):
 return tf.io.parse_single_example(example_proto, train_feature_description)
def second_parse_image_function(example_proto):
 return tf.io.parse_single_example(example_proto, test_feature_description)

In [ ]:
train_lis = []
val_lis = []
test_lis = []

for i in train_imgs:
  train_lis.append(tf.data.TFRecordDataset("/kaggle/input/tpu-getting-started/tfrecords-jpeg-192x192/train/"+i))

for i in val_imgs:
  val_lis.append(tf.data.TFRecordDataset("/kaggle/input/tpu-getting-started/tfrecords-jpeg-192x192/val/"+i))

for i in test_imgs:
  test_lis.append(tf.data.TFRecordDataset("/kaggle/input/tpu-getting-started/tfrecords-jpeg-192x192/test/"+i))

In [ ]:
train_ids = []
train_classes = []
train_images = []

val_ids = []
val_classes = []
val_images = []

test_ids = []
test_images = []

for tl in train_lis:
  temp = tl.map(_parse_image_function)
  for t in temp:
    train_classes.append(t["class"].numpy())
    train_ids.append(str(t["id"].numpy())[2:-1])
    train_images.append(t["image"].numpy())
    
for tl in val_lis:
  temp = tl.map(_parse_image_function)
  for t in temp:
    val_classes.append(t["class"].numpy())
    val_ids.append(str(t["id"].numpy())[2:-1])
    val_images.append(t["image"].numpy())

for tl in test_lis:
  temp = tl.map(second_parse_image_function)
  for t in temp:
    test_ids.append(str(t["id"].numpy())[2:-1])
    test_images.append(t["image"].numpy())

In [ ]:
class CustDat(Dataset):
    def __init__(self , images , classes , ids , transform , mode):
        self.images = images
        self.classes = classes
        self.ids = ids
        self.transform = transform
        self.mode = mode
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self , idx):
        img = Image.open(io.BytesIO(self.images[idx]))
        img = self.transform(img)
        if self.mode == "test":
            idd = self.ids[idx]
            return (img , idd)
        else:
            label = self.classes[idx]
            return (img , label)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((120 , 120)) , 
    transforms.ToTensor() , 
    transforms.Normalize((0 , 0 , 0) , (1 , 1 , 1))
])

In [ ]:
train_cust = CustDat(train_images , train_classes , None , transform , "train")
val_cust = CustDat(val_images , val_classes , None , transform , "val")
test_cust = CustDat(test_images , None , test_ids , transform , "test")

In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=1)

    def forward(self, x: Tensor) -> Tensor:
        identity = self.downsample(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = self.relu(out)

        return out


class ResNet18(nn.Module):
    def __init__(self,
                 image_size: int,
                 in_channels: int,
                 num_classes: int,
                 base_width=8,
                 ):
        super().__init__()
        self.image_size = image_size
        self.embed = nn.Conv2d(in_channels=in_channels, out_channels=base_width * 1, kernel_size=1)
        self.bn1 = nn.BatchNorm2d(base_width * 1)
        self.relu = nn.ReLU(inplace=True)
        self.block1 = BasicBlock(in_channels=base_width * 1, out_channels=base_width * 1)
        self.block2 = BasicBlock(in_channels=base_width * 1, out_channels=base_width * 2)
        self.block3 = BasicBlock(in_channels=base_width * 2, out_channels=base_width * 4)
        self.block4 = BasicBlock(in_channels=base_width * 4, out_channels=base_width * 8)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(base_width * 8, num_classes)

    def forward(self, x: Tensor) -> Tensor:
        x = self.embed(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [ ]:
temp_model = models.resnet18(pretrained=True)

In [ ]:
tot_fr = temp_model.fc.in_features

In [ ]:
temp_model.fc = nn.Linear(tot_fr , len(train_classes))

In [ ]:
temp_model = ResNet18(192, 3, len(train_classes))

In [ ]:
SERIAL_EXEC = xmp.MpSerialExecutor()
WRAPPED_MODEL = xmp.MpModelWrapper(temp_model)

In [ ]:
def run(rank):
  train_sampler = torch.utils.data.distributed.DistributedSampler(
      train_cust , 
      num_replicas = 8 , 
      rank = xm.get_ordinal() , 
      shuffle = True , 
      seed = 0
  )
  val_sampler = torch.utils.data.distributed.DistributedSampler(
      val_cust , 
      num_replicas = 8 , 
      rank = xm.get_ordinal() , 
      shuffle = True , 
      seed = 0
  )
  train_loader = DataLoader(
      train_cust , 
      batch_size = 16 , 
      sampler = train_sampler , 
      num_workers = 1 , 
      drop_last = False
  )
  val_loader = DataLoader(
      val_cust , 
      batch_size = 16 , 
      sampler = val_sampler , 
      num_workers = 1 , 
      drop_last = False
  )
  test_loader = DataLoader(
      test_cust , 
      batch_size = 16 ,
      num_workers = 1 , 
      drop_last = False
  )
  device = xm.xla_device()
  model = WRAPPED_MODEL.to(device)
  lr = 0.01 * xm.xrt_world_size()
  optimizer = SGD(model.parameters() , lr = lr)
  loss_fn = nn.CrossEntropyLoss()

  num_epochs = 10

  for epoch in range(num_epochs):

    #training
    para_loader = pl.ParallelLoader(train_loader , [device])
    train_loss = []
    train_corr = 0
    train_sam = 0
    model.train()
    for x , (data , label) in enumerate(para_loader.per_device_loader(device)):
      optimizer.zero_grad()
      output = model(data)
      loss = loss_fn(output , label)
      #accuracy
      _ , pred = torch.max(output , 1)
      train_corr += (pred == label).sum()
      train_sam += label.shape[0]
      loss.backward()
      train_loss.append(loss.item())
      xm.optimizer_step(optimizer)
      
    #evaluation
    model.eval()
    val_loss = []
    val_corr = 0
    val_sam = 0
    with torch.no_grad():
      para_loader = pl.ParallelLoader(val_loader , [device])
      for x , (data , label) in enumerate(para_loader.per_device_loader(device)):
        output = model(data)
        loss = loss_fn(output , label)
        #accuracy
        _ , pred = torch.max(output , 1)
        val_corr += (pred == label).sum()
        val_sam += label.shape[0]
        val_loss.append(loss.item())

    t_ac = 100.0 * train_corr / train_sam
    v_ac = 100.0 * val_corr / val_sam
    t_lo = torch.sum(torch.Tensor(train_loss))
    v_lo = torch.sum(torch.Tensor(val_loss))
    
    print("epoch is ",epoch," train accu ",t_ac," train loss ",t_lo," val accu ",v_ac," val loss ",v_lo)
  
  if xm.is_master_ordinal():
    dic = {}
    model.eval()
    with torch.no_grad():
      para_loader = pl.ParallelLoader(test_loader , [device])
      for x , (data , ids) in enumerate(para_loader.per_device_loader(device)):
        output = model(data)
        _ , pred = torch.max(output , 1)
        for i in range(pred.shape[0]):
          dic[ids[i]] = int(pred[i].cpu().detach().numpy())
      df = pd.DataFrame(dic.items())
      df.to_csv("fin_sub.csv")

In [ ]:
xmp.spawn(run , nprocs = 8 , start_method = "fork")

In [ ]:
os.listdir()

In [ ]:
dff = pd.read_csv('fin_sub.csv')
dff.head()

In [ ]:
fin = pd.DataFrame({"id":dff["0"].values , "label":dff["1"].values})

In [ ]:
fin.to_csv('submission.csv' , index = False)

In [ ]:
os.listdir()

In [ ]:
os.remove("fin_sub.csv")

In [ ]:
fin.head()